In [25]:
import pandas as pd
import numpy as np
from numpy.core.records import fromarrays
from scipy.io import savemat

In [26]:
choice = 0  #start from 0
devices = ['nfet_01v8_lvt', 'nfet_01v8', 'pfet_01v8_lvt', 'pfet_01v8']

# widths used for characterization and fringe cap parameters (fringe caps are not included in ngspice output)
w = np.array([5, 5, 5, 5])
nfing = np.array([1, 1, 1, 1])
cgdo_w = np.array([2.39289e-10, 2.44907e-10, 2e-11, 5.24893e-11])*1e-6

In [27]:
# read ngspice data
df_raw = pd.read_csv('./simulation/techsweep_'+devices[choice]+'.txt', sep='\s+')
par_names = df_raw.columns.to_list()
fet_name = par_names[1].split('[')[0]

# remove unwanted columns and rename for readability
df = df_raw.drop(['frequency', 'frequency.1'], axis=1)
df = df.apply(pd.to_numeric)
df.columns = df.columns.str.replace(fet_name, '')
df.columns = df.columns.str.replace(fet_name[1:], '')
df.columns = df.columns.str.replace('onoise..', 'n')
df.columns = df.columns.str.removeprefix('[')
df.columns = df.columns.str.removesuffix(']')

# round sweep vectors to easily addressable values
df['l'] = df['l'].apply(lambda x: round(x/1e-6, 3))
df['vgs'] = df['vgs'].apply(lambda x: round(x, 3))
df['vds'] = df['vds'].apply(lambda x: round(x, 3))
df['vbs'] = df['vbs'].apply(lambda x: round(x, 3))
df

,capbd,capbs,cdd,cgb,cgd,cgg,cgs,css,gds,gm,gmbs,id,l,vbs,vds,vgs,vth,n1overf,nid
0,2.820000e-15,2.820000e-15,4.190000e-16,-1.460000e-15,-3.334000e-16,1.619000e-15,1.744000e-16,-8.537000e-17,6.077000e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.15,0.0,0.000,0.0,0.6197,0.000000e+00,2.389000e-16
1,2.508000e-15,2.508000e-15,3.248000e-16,-1.327000e-15,-2.635000e-16,1.448000e-15,1.426000e-16,-7.198000e-17,2.081000e-11,0.000000e+00,0.000000e+00,0.000000e+00,0.15,-0.2,0.000,0.0,0.6478,0.000000e+00,8.181000e-17
2,2.284000e-15,2.284000e-15,2.656000e-16,-1.223000e-15,-2.196000e-16,1.321000e-15,1.219000e-16,-6.206000e-17,9.059000e-12,0.000000e+00,0.000000e+00,0.000000e+00,0.15,-0.4,0.000,0.0,0.6709,0.000000e+00,3.561000e-17
3,2.774000e-15,2.820000e-15,7.575000e-17,-1.466000e-15,-5.426000e-17,1.596000e-15,-7.605000e-17,7.673000e-17,3.319000e-11,2.992000e-11,6.991000e-12,1.162000e-12,0.15,0.0,0.025,-0.0,0.6185,1.576000e-16,2.618000e-16
4,2.476000e-15,2.508000e-15,5.559000e-17,-1.330000e-15,-4.025000e-17,1.430000e-15,-5.962000e-17,5.604000e-17,1.159000e-11,1.061000e-11,1.877000e-12,4.013000e-13,0.15,-0.2,0.025,-0.0,0.6464,5.429000e-17,9.001000e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255787,1.525000e-15,2.508000e-15,9.331000e-17,3.032000e-15,-7.865000e-17,8.953000e-14,-9.248000e-14,5.475000e-14,5.449000e-06,4.495000e-04,9.444000e-05,3.357000e-04,3.00,-0.2,1.775,1.8,0.4874,2.721000e-08,4.104000e-12
255788,1.477000e-15,2.284000e-15,9.463000e-17,2.154000e-15,-8.297000e-17,8.839000e-14,-9.047000e-14,5.385000e-14,5.115000e-06,4.305000e-04,7.798000e-05,3.185000e-04,3.00,-0.4,1.775,1.8,0.5237,2.592000e-08,3.880000e-12
255789,1.571000e-15,2.820000e-15,8.398000e-17,4.368000e-15,-6.748000e-17,9.083000e-14,-9.513000e-14,5.593000e-14,5.747000e-06,4.765000e-04,1.198000e-04,3.570000e-04,3.00,-0.0,1.800,1.8,0.4469,2.878000e-08,4.443000e-12
255790,1.518000e-15,2.508000e-15,8.618000e-17,3.032000e-15,-7.262000e-17,8.953000e-14,-9.248000e-14,5.475000e-14,5.304000e-06,4.498000e-04,9.449000e-05,3.358000e-04,3.00,-0.2,1.800,1.8,0.4874,2.722000e-08,4.106000e-12


In [28]:
# sweep variable vectors
l =   np.unique(df['l'])
vgs = np.unique(df['vgs'])
vds = np.unique(df['vds'])
vsb = np.unique(-df['vbs'])

In [29]:
# ngspice sweep order is l, vgs, vds, vsb
dims = [len(l), len(vgs), len(vds), len(vsb)]
id = np.reshape(df['id'].values, dims)
vt = np.reshape(df['vth'].values, dims)
gm = np.reshape(df['gm'].values, dims)
gmb = np.reshape(df['gmbs'].values, dims)
gds = np.reshape(df['gds'].values, dims)
cfringe = w[choice]*cgdo_w[choice]
cgg = np.reshape(df['cgg'].values, dims) \
      + 2*cfringe
cgb = -np.reshape(df['cgb'].values, dims)
cgd = -np.reshape(df['cgd'].values, dims) \
      + cfringe
cgs = -np.reshape(df['cgs'].values, dims) \
      + cfringe
cdd = np.reshape(df['cdd'].values, dims) \
      + np.reshape(df['capbd'].values, dims) \
      + cfringe
css = np.reshape(df['css'].values, dims) \
      + np.reshape(df['capbd'].values, dims) \
      + cfringe
sth = np.reshape(df['nid'].values, dims)**2
sfl = np.reshape(df['n1overf'].values, dims)**2


In [30]:
dic = {
  "INFO": "SkyWater, 130nm CMOS, BSIM4",
  "CORNER": "NOM",
  "TEMP": 300.0,
  "VGS": vgs,
  "VDS": vds,
  "VSB": vsb,
  "L": l,
  "W": w[choice],
  "NFING": nfing[choice],
  "ID": id,
  "VT": vt,
  "GM": gm,
  "GMB": gmb,
  "GDS": gds,
  "CGG": cgg,
  "CGB": cgb,
  "CGD": cgd,
  "CGS": cgs,
  "CDD": cdd,
  "CSS": css,
  "STH": sth,
  "SFL": sfl
}
savemat('./simulation/'+devices[choice]+'.mat', {devices[choice]: dic})